In [1]:
import mmcv
import matplotlib.pyplot as plt
import math 
from matplotlib.patches import Rectangle, Polygon
from PIL import Image
import numpy as np
import os
import shutil
import json
import cv2
from tqdm import tqdm

/home/kites/miniconda3/envs/env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
def npxywha2vertex(box):
    """
    use radian
    X=x*cos(a)-y*sin(a)
    Y=x*sin(a)+y*cos(a)
    """
    batch = box.shape[0]

    center = box[:,:2]
    w = box[:,2]
    h = box[:,3]
    rad = box[:,4]

    # calculate two vector
    verti = np.empty((batch,2), dtype=np.float32)
    verti[:,0] = (h/2) * np.sin(rad)
    verti[:,1] = - (h/2) * np.cos(rad)

    hori = np.empty((batch,2), dtype=np.float32)
    hori[:,0] = (w/2) * np.cos(rad)
    hori[:,1] = (w/2) * np.sin(rad)

    tl = center + verti - hori
    tr = center + verti + hori
    br = center - verti + hori
    bl = center - verti - hori

    return np.concatenate([tl,tr,br,bl], axis=1)

In [18]:
path = "../../../data/Phamacity/"

In [19]:
def json2txtAnnotations(path, outputPath):
    for filename in os.listdir(path):
        # print(filename)
        if filename.endswith(".json"):
            f = open(path + filename)
            data = json.load(f)
            for anno in data['annotations']: 
                # print(anno)
                image_id = anno['image_id']
                bbox = anno['bbox']
                bbox[4] = bbox[4] / 180 * math.pi
                bbox = np.array(bbox)
                bbox = npxywha2vertex(bbox[np.newaxis])
                with open(outputPath + image_id+'.txt', 'w') as txt:
                    for cor in bbox[0]:
                        txt.write(cor.astype('str') + ' ')
                    txt.write('person 0\n')
            f.close()
        

In [20]:
def png2jpg(path, outputPath):
    for dir in os.listdir(path):
        if os.path.isdir(path+dir):
            print(dir)
            for filename in tqdm(os.listdir(path+dir)):
                if filename.endswith(".jpg"):
                    portion = os.path.splitext(filename)
                    img = cv2.imread(path+dir+'/'+filename)
                    cv2.imwrite(outputPath + portion[0] + '.png', img)
                    # shutil.copy2(path+dir+'/'+filename, path+'images')
                

In [27]:
json2txtAnnotations(path, "../../../data/PhamacityDota/anno/")

In [24]:
png2jpg(path, "../../../data/PhamacityDota/images/")

100%|███████████████████████████████████████████████████████████████| 688/688 [00:47<00:00, 14.47it/s]
